In [2]:
import pandas as pd
import numpy as np

from datasets import load_dataset


/root/TwoTowerSearch/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
train_dataset = load_dataset('ms_marco', 'v2.1', split='train')
test_dataset = load_dataset('ms_marco', 'v2.1', split='validation')

In [22]:
# view structure of the passages column in the dataset
texts = test_dataset[1321]
texts


{'answers': ['The average temperature of Brooklyn, New York is 52.78°F.'],
 'passages': {'is_selected': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
  'passage_text': ['Currently in Brooklyn as of . 42°(Feels like 39°) Scattered Clouds; Wind: @ 5 mph; Barometer: 30.38 in. Sunrise: 06:53:34 am; Sunset: 05:26:58 pm; Humidity: 49; Visibility: 10 mi. UV Index: 2',
   'News 12 Brooklyn Weather Center: Hyper-local weather, live radar, dual Doppler and more. News 12 Brooklyn Weather Center: Hyper-local weather, live radar, dual Doppler and more. Home',
   'Home » Local » Weather Report Brooklyn, New York Weather Report · Interactive Map · Extended Forecast · Hourly Forecast · Past Observations · Historic Averages',
   "When we are back up and running you will be prompted at that time to complete your sign in. Until then, enjoy the local news, weather, traffic and more that's as local as local news gets.. back to site.",
   "Intellicast.com: The Authority in Expert Weather Brooklyn, New York weather condit

In [ ]:

from data_prep import create_triplets

# Generate triplets
train_triplets_stage1 = create_triplets(train_dataset, include_hard_negatives=False)
#train_triplets_stage2 = create_triplets(train_dataset, include_hard_negatives=True)

test_triplets = create_triplets(test_dataset, include_hard_negatives=False)

# Print samples from both train and test sets
print("TRAINING TRIPLETS (stage 1):")
for t in train_triplets_stage1[:10]:
    print(f"Query: {t[0]}\nPositive: {t[1]}\nNegative: {t[2]}\n{'-'*40}")

#print("\nTRAINING TRIPLETS (stage 2):")
#for t in train_triplets_stage2[:10]:
#    print(f"Query: {t[0]}\nPositive: {t[1]}\nNegative: {t[2]}\n{'-'*40}")

print("\nTEST TRIPLETS:")
for t in test_triplets[:3]:
    print(f"Query: {t[0]}\nPositive: {t[1]}\nNegative: {t[2]}\n{'-'*40}")

TRAINING TRIPLETS (stage 1):
Query: )what was the immediate impact of the success of the manhattan project?
Positive: The presence of communication amid scientific minds was equally important to the success of the Manhattan Project as scientific intellect was. The only cloud hanging over the impressive achievement of the atomic researchers and engineers is what their success truly meant; hundreds of thousands of innocent lives obliterated.
Negative: If you need help or if you know someone who has a problem with amphetamines, call our helpline at 1-800-943-0566. 1  Related Articles. 2  Find Help Today. 3  Amphetamine Street Names: Terms You Need to Know! - The most common amphetamine street name is speed but it is important to educate yourself about the rest.
----------------------------------------
Query: _________ justice is designed to repair the harm to victim, the community and the offender caused by the offender criminal act. question 19 options:
Positive: The approach is based on

In [26]:
print(len(train_triplets_stage1), "training triplets - stage 1")
#print(len(train_triplets_stage2), "training triplets - stage 2")
print(len(test_triplets), "test triplets")

502939 training triplets - stage 1
55578 test triplets


In [27]:
from data_prep import save_triplets_to_json

# Save triplets
save_triplets_to_json(train_triplets_stage1, "train_triplets_stage1.json")
#save_triplets_to_json(train_triplets_stage2, "train_triplets_stage2.json")
save_triplets_to_json(test_triplets, "test_triplets.json")

In [8]:
from data_prep import save_passages_to_file

# Combine both train, test, and validation passages to get all passages in one list
train_passages = [passage for row in train_dataset for passage in row['passages']['passage_text']]
test_passages = [passage for row in test_dataset for passage in row['passages']['passage_text']]
all_passages = train_passages + test_passages
print(len(all_passages), "total passages")

# Save to a JSON file
save_passages_to_file(all_passages, 'msmarco_v2_docs.json')


9078734 total passages
Passages saved to msmarco_v2_docs.json


### Retrieving texts for database creation

In [ ]:
# load triplets from v1 dataset to be used for database
database_dataset_train = load_dataset('ms_marco', 'v1.1', split='train')
database_dataset_test = load_dataset('ms_marco', 'v1.1', split='test')
database_dataset_val = load_dataset('ms_marco', 'v1.1', split='validation')


In [9]:
from data_prep import save_passages_to_file

# Combine both train, test, and validation passages to get all passages in one list
database_passages_train = [passage for row in database_dataset_train for passage in row['passages']['passage_text']]
database_passages_test = [passage for row in database_dataset_test for passage in row['passages']['passage_text']]
database_passages_val = [passage for row in database_dataset_val for passage in row['passages']['passage_text']]
all_passages = database_passages_train + database_passages_test + database_passages_val
print(len(all_passages), "total passages")

# Save to a JSON file
save_passages_to_file(all_passages, 'msmarco_v1_docs.json')


837729 total passages
Passages saved to msmarco_v1_docs.json


In [37]:
for i, passage in enumerate(all_passages[:15]):
    print(f"Passage {i+1}: {passage}\n{'-'*40}")

Passage 1: Since 2007, the RBA's outstanding reputation has been affected by the 'Securency' or NPA scandal. These RBA subsidiaries were involved in bribing overseas officials so that Australia might win lucrative note-printing contracts. The assets of the bank include the gold and foreign exchange reserves of Australia, which is estimated to have a net worth of A$101 billion. Nearly 94% of the RBA's employees work at its headquarters in Sydney, New South Wales and at the Business Resumption Site.
----------------------------------------
Passage 2: The Reserve Bank of Australia (RBA) came into being on 14 January 1960 as Australia 's central bank and banknote issuing authority, when the Reserve Bank Act 1959 removed the central banking functions from the Commonwealth Bank. The assets of the bank include the gold and foreign exchange reserves of Australia, which is estimated to have a net worth of A$101 billion. Nearly 94% of the RBA's employees work at its headquarters in Sydney, New S

Loaded 755369 passages from all_docs.json
["Since 2007, the RBA's outstanding reputation has been affected by the 'Securency' or NPA scandal. These RBA subsidiaries were involved in bribing overseas officials so that Australia might win lucrative note-printing contracts. The assets of the bank include the gold and foreign exchange reserves of Australia, which is estimated to have a net worth of A$101 billion. Nearly 94% of the RBA's employees work at its headquarters in Sydney, New South Wales and at the Business Resumption Site.", "The Reserve Bank of Australia (RBA) came into being on 14 January 1960 as Australia 's central bank and banknote issuing authority, when the Reserve Bank Act 1959 removed the central banking functions from the Commonwealth Bank. The assets of the bank include the gold and foreign exchange reserves of Australia, which is estimated to have a net worth of A$101 billion. Nearly 94% of the RBA's employees work at its headquarters in Sydney, New South Wales and a